In [1]:
# !git pull

In [8]:
# https://github.com/huseinzol05/Malay-Dataset/tree/master/speech/sebut-perkataan
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech-bahasa.zip
# !mkdir sebut-perkataan
# !unzip speech-bahasa.zip -d sebut-perkataan

In [9]:
import malaya_speech.train as train
import malaya_speech

In [10]:
encoded = malaya_speech.char.encode('hello ketiak saya masham')
encoded

[106,
 103,
 110,
 110,
 113,
 34,
 109,
 103,
 118,
 107,
 99,
 109,
 34,
 117,
 99,
 123,
 99,
 34,
 111,
 99,
 117,
 106,
 99,
 111,
 1]

In [11]:
from glob import glob
import os

files = glob('sebut-perkataan/*/*.wav', recursive = True)
len(files)

1463

In [25]:
def get_text(file):
    file = file.replace('.wav', '')
    splitted = file.split('/')[1:]
    splitted[0] = splitted[0].replace('-woman', '').replace('-man', '').replace('-', ' ')
    return ' '.join(splitted).lower().strip()
    


'sebut perkataan amko'

In [30]:
audios, texts = [], []

for file in files:
    
    text = get_text(file)
    audios.append(file)
    texts.append(text)

    
len(audios), len(texts)

(1463, 1463)

In [34]:
from tqdm import tqdm

def generator():
    for i in tqdm(range(len(audios))):
        wav_data, sr = malaya_speech.load(audios[i])

        yield {
            'waveforms': wav_data.tolist(),
            'waveform_lens': [len(wav_data)],
            'targets': malaya_speech.char.encode(texts[i]),
            'raw_transcript': [texts[i]],
        }
        
generator = generator()

In [35]:
import os
import tensorflow as tf

os.system('rm tolong-sebut/data/*')
DATA_DIR = os.path.expanduser('tolong-sebut/data')
tf.gfile.MakeDirs(DATA_DIR)

In [36]:
shards = [{'split': 'train', 'shards': 99}, {'split': 'dev', 'shards': 1}]

In [37]:
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'tolong-sebut')

  0%|          | 0/1463 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


100%|██████████| 1463/1463 [00:24<00:00, 60.74it/s]


INFO:tensorflow:Generated 1463 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`



INFO:tensorflow:Data shuffled.
